In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem import AllChem

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import PredefinedSplit, GridSearchCV

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_hi_metrics

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/hi/sol/train_1.csv')
test = pd.read_csv('../../../../data/hi/sol/test_1.csv')

train

,Unnamed: 0,smiles,value
0,2089,O=C(CN1C(=O)CCC1=O)Nc1ccc(F)c(F)c1F,False
1,2099,COc1ccc(-c2nnc(NC(=O)c3ccc4ccccc4c3)o2)cc1OC,True
2,61,O=C(Nc1ccc2ccccc2n1)c1ccc(N2CCOC2=O)cc1,True
3,89,Cc1c[nH]c(=O)n1-c1ccc(C(=O)Nc2ccc3ccccc3n2)cc1,True
4,97,O=C(C1CCC(O)CC1)N1CCCN(c2ccc(F)cc2)CC1,False
...,...,...,...
1437,2143,N#CCCn1nc(C(F)(F)F)cc1O,False
1438,2147,Cc1ccccc1N1C(=O)c2ccc(C(=O)NC(C)(C)C)cc2C1=O,False
1439,2160,CC(C)(C)C(=O)Nc1sc2c(c1C#N)CCC2,True
1440,2165,C[C@@H](c1ccc(F)cc1)n1nnc2cnc3ccc(-c4ccc5ocnc5...,True


In [4]:
from scipy.spatial.distance import jaccard


def run_knn_gridsearch_tanimoto(train_fps, test_fps):
    split_index = [-1] * len(train_fps) + [0] * len(test_fps)
    pds = PredefinedSplit(test_fold = split_index)

    X = train_fps + test_fps
    y = train['value'].to_list() + test['value'].to_list()

    params = {
        'n_neighbors': [3, 5, 7, 10, 12, 15],
        'weights': ['uniform', 'distance'],
    }
    knn = KNeighborsClassifier(metric=jaccard)

    grid_search = GridSearchCV(knn, params, cv=pds, refit=False, scoring='average_precision', verbose=3)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    knn = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], weights=best_params['weights'], metric=jaccard)
    knn.fit(train_fps, train['value'])

    train_preds = knn.predict_proba(train_fps)[:, 1]
    train_metrics = get_hi_metrics(train, train_preds)

    test_preds = knn.predict_proba(test_fps)[:, 1]
    test_metrics = get_hi_metrics(test, test_preds)
    return train_metrics, test_metrics


In [5]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]

train_metrics, test_metrics = run_knn_gridsearch_tanimoto(train_maccs_fps, test_maccs_fps)
print(train_metrics)
print(test_metrics)

Fitting 1 folds for each of 12 candidates, totalling 12 fits
[CV 1/1] END ....n_neighbors=3, weights=uniform;, score=0.315 total time=   5.3s
[CV 1/1] END ...n_neighbors=3, weights=distance;, score=0.335 total time=   5.2s
[CV 1/1] END ....n_neighbors=5, weights=uniform;, score=0.339 total time=   5.3s
[CV 1/1] END ...n_neighbors=5, weights=distance;, score=0.369 total time=   5.2s
[CV 1/1] END ....n_neighbors=7, weights=uniform;, score=0.359 total time=   5.2s
[CV 1/1] END ...n_neighbors=7, weights=distance;, score=0.392 total time=   5.2s
[CV 1/1] END ...n_neighbors=10, weights=uniform;, score=0.388 total time=   5.2s
[CV 1/1] END ..n_neighbors=10, weights=distance;, score=0.409 total time=   5.2s
[CV 1/1] END ...n_neighbors=12, weights=uniform;, score=0.385 total time=   5.2s
[CV 1/1] END ..n_neighbors=12, weights=distance;, score=0.410 total time=   5.2s
[CV 1/1] END ...n_neighbors=15, weights=uniform;, score=0.393 total time=   5.2s
[CV 1/1] END ..n_neighbors=15, weights=distance;

# Final Evaluation

In [6]:
def fit_predict(train, test):
    train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
    train_morgan_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]

    test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
    test_morgan_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]

    knn = KNeighborsClassifier(n_neighbors=15, weights='distance', metric=jaccard)
    knn.fit(train_morgan_fps, train['value'])

    train_result = train.copy()
    train_result['preds'] = knn.predict_proba(train_morgan_fps)[:, 1]

    test_result = test.copy()
    test_result['preds'] = knn.predict_proba(test_morgan_fps)[:, 1]

    return train_result, test_result


In [7]:
for i in [1, 2, 3]:
    train = pd.read_csv(f'../../../../data/hi/sol/train_{i}.csv')
    test = pd.read_csv(f'../../../../data/hi/sol/test_{i}.csv')

    train_preds, test_preds = fit_predict(train, test)
    train_preds.to_csv(f'../../../../predictions/hi/sol/knn_maccs/train_{i}.csv')
    test_preds.to_csv(f'../../../../predictions/hi/sol/knn_maccs/test_{i}.csv')
